In [1]:
import pandas as pd
from collections import Counter
import nltk
# Tar bort varningarna för ett fel som vi kan ignorera
pd.options.mode.chained_assignment = None  # default='warn'

# Your Data Mining Pilot Study and Plan

## Background

*Describe the organization and its operations, operational environment, analytical needs, etc.*

*Replace the contents of this cell with your own text.*

## Problem description

*Describe the problem that the organization is facing and the research question it needs answered.*

*Replace the contents of this cell with your own text.*

## Data collection

*Document your data collection process and the properties of the data here. Implement, using Python code, to load and preprocess your selected dataset.*

*Replace the contents of this cell with your own text.*

In [2]:
# Läser in vår data från csv filen
stackOverflow = pd.read_csv(
    './Data.csv',
    encoding='utf-8'
)

# Skriver ut storleken på vårt dataset
print('Number of Rows and Columns:')
print(stackOverflow.shape)

# Printar ut namnen på att columns som finns i datasetet
listOfColumnNames = stackOverflow.columns.values.tolist()
print('\nColumn names:')
for name in listOfColumnNames:
    print(name)

# Ändra här för att ta med fler columns in i dataFramen
df = stackOverflow[['AcceptedAnswerId', 'Title',
                    'CreationDate', 'Body']]

# Denna rad funkar inte men det som ska göras här är att om raden för AcceptedAnswerId inte är tom så ska raden bort då vi letar efter frågor utan
# ett accepterat svar
#df = df.loc[df['AcceptedAnswerId'] != '']

# Printar ut från vilka datum som vår data kommer ifrån
minValue = df['CreationDate'].min()
maxValue = df['CreationDate'].max()
print('\nDatum mellan vilka datan är tagen ifrån: ' +
      minValue + ' & ' + maxValue + '\n')


# Sätter värdena i title och body till lowercase
df['Title'] = df['Title'].str.lower()
df['Body'] = df['Body'].str.lower()


# Denna funkar men vi måste göra den mer sofistikerad då meningarna ofta hänger ihop med html taggarna och därmed inte tas bort
# Därför jag tex skrivit '<p>i' här nere för att visa att det funkar men detta måste kunna göras på ett bättre sätt.
# Kanske börja med att köra igenom koden och lägga till ett space efter och innan varje html tagg?
stopWords = ['code', 'gt', 'i', '<p>i']

# Tar bort alla stopWords från Body
df['Body'] = df['Body'].apply(lambda x: ' '.join(
    [word for word in x.split() if word not in (stopWords)]))

# Print för att testa stopWords funktionen
print('Testa StopWords funktionen: \n')
print(df['Body'].head(4) + '\n')

# Denna skapar en lista över de vanligaste orden. Du ändrar hur många ord den ska ta med genom variabeln här under
nrOfWords = 10
rslt = Counter(' '.join(df['Body']).split()).most_common(nrOfWords)

# Printar ut listan med de vanligaste orden
print('\n')
for word in rslt:
    print('{} = {}'.format(word[1], word[0]))


Number of Rows and Columns:
(117527, 23)

Column names:
Id
PostTypeId
AcceptedAnswerId
ParentId
CreationDate
DeletionDate
Score
ViewCount
Body
OwnerUserId
OwnerDisplayName
LastEditorUserId
LastEditorDisplayName
LastEditDate
LastActivityDate
Title
Tags
AnswerCount
CommentCount
FavoriteCount
ClosedDate
CommunityOwnedDate
ContentLicense

Datum mellan vilka datan är tagen ifrån: 2019-09-22 00:02:14 & 2020-11-08 04:47:27

Testa StopWords funktionen: 

0    use load / unload assembly to get all types th...
1    cannot figure out what isnt being set for the ...
2    need the following data</p> <p><a href="https:...
3    <pre><code>using system; using system.collecti...
Name: Body, dtype: object


721941 = the
643061 = =
540045 = {
473961 = }
473621 = to
306589 = a
267119 = is
240393 = in
228562 = and
215552 = public


## Data analysis

*Document you choice and motivation for selected data mining method(s) here. Choose a data mining method(s) to use in Python code to perform an analysis of your chosen dataset. Describe why you chose the method(s) and what interesting things you have found from the analysis.*

*Replace the contents of this cell with your own text.*

## Evaluation of results

*Document an evaluation your analysis results and describe how potentially actionable they are.*

*Replace the contents of this cell with your own text.*

In [3]:
# Add your own code

## Schedule and description of project plan

*Rough schedule for the project beyond the pilot study presented in 3-5. This does not have to be advanced, you can simply provide an estimate based upon reported schedules for similar projects in the literature.*

*Replace the contents of this cell with your own text.*

## Ethical aspects that need to be considered

*Are there ethical aspects that need to be considered? Are there legal implications (e.g., personal data / GDPR)? Are there implications if the case organization is a business, public authority, or nonprofit entity?*

*Replace the contents of this cell with your own text.*